In [1]:
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold # import KFold
from sklearn import metrics

**Connect to drive**

In [3]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/MyDrive/CA683')

Mounted at /content/drive/


**Load the merged data**

In [5]:
import matplotlib.pyplot as plt

data =pd.read_csv("bitcoin_and_sentiments_data.csv")
data.columns
data.fillna(inplace=True, value=0)

data.head(2)

,Unnamed: 0,time,closed_price_by_hour,price_diff,movement,Daily_Weight_mean_by_hour,Daily_Weight_count_by_hour,blob_sent_mean_by_hour,subjectivity_mean_by_hour,vader_sent_mean_by_hour,tweet_diff,blob_sent_mean_by_hour_diff,vader_sent_mean_by_hour_diff,tweet_movement,blob_sent_movement,vader_sent_movement
0,0,2016-02-19 08:00:00,418.409535,1.745839,1,0.0,1,0.0,0.15,-0.4019,0.0,-3.93183,-0.442529,1,0,0
1,1,2016-03-28 03:00:00,423.431613,0.030249,1,0.0,2,0.0,0.00,0.0000,1.0,0.00000,0.401900,1,1,1


**Define the features**

In [6]:
vader = data["vader_sent_mean_by_hour"].values
blob = data["blob_sent_mean_by_hour"].values
tweetdir = data["tweet_diff"].values
tweet_vol = data["Daily_Weight_count_by_hour"].values
v_move = data["vader_sent_mean_by_hour_diff"].values
b_move = data["blob_sent_mean_by_hour_diff"].values
diff = data['movement'].values
multifeat = data.filter(["vader_sent_mean_by_hour","blob_sent_mean_by_hour","tweet dir","Daily_Weight_count_by_hour"])
close = data['closed_price_by_hour'].values

In [7]:
features = [multifeat,vader,blob,tweetdir,tweet_vol,v_move,b_move]

feature = ['multifeat','vader','blob','tweetdir','tweet_vol','v_move','b_move']

In [8]:
def f1(cm):
    aNo = cm[0][0]
    ayes = cm[1][0]
    pno = cm[0][1]
    pyes = cm[1][1]
    

**Kfold Cross Validation with Logistic Regression**

In [9]:
#Kfold Cross Validation with Logistic Regression

for i in range(len(features)):
    if i >0:
        xbar = features[i].reshape(-1,1)
    else:
        xbar = features[i]
    kf = KFold(n_splits=10) # Define the split - into 2 folds 
    
    kf.get_n_splits(xbar) # returns the number of splitting iterations in the cross-validator
    #KFold(n_splits=10, random_state=None, shuffle=False)
    scores = []
    f1=[]
    mat = np.zeros([2,2])
    for train_index, test_index in kf.split(xbar):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = xbar[train_index[0]:train_index[1189]], xbar[test_index[0]:test_index[132]]
        y_train, y_test = diff[train_index[0]:train_index[1189]], diff[test_index[0]:test_index[132]]
        logisticRegr = LogisticRegression()
        logisticRegr.fit(X_train, y_train)

        predictions = logisticRegr.predict(X_test)
        score = logisticRegr.score(X_test, y_test)
        scores.append(score)
        #print("{} Score : {}".format(feature[i],score))
        cm = metrics.confusion_matrix(y_test, predictions)
        f1_score = metrics.f1_score(y_test,predictions)
        f1.append(f1_score)
        mat += cm
        
        #print(cm)
    idx = np.mean(scores)
    print("{} Score : {}".format(feature[i],round(idx, 3)))
    print(round(np.mean(f1), 3))
    print(mat)



multifeat Score : 0.536
0.691
[[ 19. 584.]
 [ 28. 689.]]
vader Score : 0.542
0.696
[[ 19. 584.]
 [ 21. 696.]]
blob Score : 0.542
0.7
[[  8. 595.]
 [  9. 708.]]
tweetdir Score : 0.539
0.696
[[ 12. 591.]
 [ 18. 699.]]
tweet_vol Score : 0.543
0.703
[[  0. 603.]
 [  0. 717.]]
v_move Score : 0.545
0.7
[[ 19. 584.]
 [ 16. 701.]]
b_move Score : 0.545
0.692
[[ 41. 562.]
 [ 39. 678.]]


In [ ]:
print(diff)

[1 0 1 ... 0 0 1]


**Multi feature logistic regression**

In [10]:
#multi feature logistic regression

features = [multifeat, vader,blob,tweetdir,tweet_vol]

feature = ['multifeat', 'vader','blob','tweet_dir','tweet_volume']

scores = []
f1=[]
mat = np.zeros([2,2])
for i in range(len(features)):
  df_label = pd.DataFrame(diff)
  if i == 0:
    X_train, X_test, y_train, y_test = train_test_split(multifeat, df_label, test_size=0.20, random_state=42)
  else:
    df_feature = pd.DataFrame(features[i])

    X_train, X_test, y_train, y_test = train_test_split(df_feature, df_label, test_size=0.20, random_state=42)
    #print(y_train)
  
  logisticRegr = LogisticRegression()
  logisticRegr.fit(X_train, y_train)
  predictions = logisticRegr.predict(X_test)
  score = logisticRegr.score(X_test, y_test)
  scores.append(score)
  #print("{} Score : {}".format(feature[i],score))
  cm = metrics.confusion_matrix(y_test, predictions)
  f1_score = metrics.f1_score(y_test,predictions)
  f1.append(f1_score)
  mat += cm
  
  #print(cm)
  idx = np.mean(scores)
  print("{} Score : {}".format(feature[i],round(idx, 3)))
  print("{} F1 Score : {}".format(feature[i],round(np.mean(f1), 3)))
  print(mat)

multifeat Score : 0.539
multifeat F1 Score : 0.699
[[  1. 214.]
 [  0. 249.]]
vader Score : 0.539
vader F1 Score : 0.699
[[  2. 428.]
 [  0. 498.]]
blob Score : 0.538
blob F1 Score : 0.699
[[  2. 643.]
 [  0. 747.]]
tweet_dir Score : 0.537
tweet_dir F1 Score : 0.698
[[  2. 858.]
 [  2. 994.]]
tweet_volume Score : 0.537
tweet_volume F1 Score : 0.698
[[   2. 1073.]
 [   2. 1243.]]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [ ]:
data.head(2)

,Unnamed: 0,time,closed_price_by_hour,price_diff,movement,Daily_Weight_mean_by_hour,Daily_Weight_count_by_hour,blob_sent_mean_by_hour,subjectivity_mean_by_hour,vader_sent_mean_by_hour,tweet_diff,blob_sent_mean_by_hour_diff,vader_sent_mean_by_hour_diff,tweet_movement,blob_sent_movement,vader_sent_movement
0,0,2016-02-19 08:00:00,418.409535,1.745839,1,0.0,1,0.000000,0.15,-0.4019,0.0,-3.931830,-0.442529,1,0,0
1,1,2016-05-15 21:00:00,456.989825,-0.042136,0,1.0,1,0.136364,0.50,0.0000,0.0,0.136364,0.401900,1,1,1


**KNN approach**

In [11]:
from sklearn import neighbors
scores = []
k_neighbours = []
feat = data.filter(['closed_price_by_hour','vader_sent_mean_by_hour']).values
for i in range(0,len(features)):
    df_feature = pd.DataFrame(features[i])
    df_label = pd.DataFrame(diff)
    for j in range(3,6):
        for weights in ["distance"]:
            print(weights,j)
            print("Working on {}".format(feature[i]))
            if i == 0:
              x_train, x_test, y_train, y_test = train_test_split(multifeat, df_label, test_size=0.20, random_state=0)
            #sbar = features[i].reshape(-1,1)
            else:    
              x_train, x_test, y_train, y_test = train_test_split(df_feature, df_label, test_size=0.20, random_state=0)
            knn = neighbors.KNeighborsClassifier(j, weights=weights)
            knn.fit(x_train, y_train)
            predictions = knn.predict(x_test)
            score = knn.score(x_test,y_test)
            print("The score is : {}".format(round(score, 3)))
            cm = metrics.confusion_matrix(y_test,predictions)
            print(cm)
            f1 =  metrics.f1_score(y_test,predictions)
            print("The F1 score is : {} \n".format(round(f1, 3)))

distance 3
Working on multifeat
The score is : 0.496
[[147  69]
 [165  83]]
The F1 score is : 0.415 

distance 4
Working on multifeat
The score is : 0.491
[[148  68]
 [168  80]]
The F1 score is : 0.404 

distance 5
Working on multifeat
The score is : 0.502
[[146  70]
 [161  87]]
The F1 score is : 0.43 

distance 3
Working on vader
The score is : 0.481
[[175  41]
 [200  48]]
The F1 score is : 0.285 

distance 4
Working on vader
The score is : 0.485
[[175  41]
 [198  50]]
The F1 score is : 0.295 

distance 5
Working on vader
The score is : 0.487
[[175  41]
 [197  51]]
The F1 score is : 0.3 

distance 3
Working on blob
The score is : 0.537
[[ 32 184]
 [ 31 217]]
The F1 score is : 0.669 

distance 4
Working on blob
The score is : 0.489
[[161  55]
 [182  66]]
The F1 score is : 0.358 

distance 5
Working on blob


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed

The score is : 0.476
[[151  65]
 [178  70]]
The F1 score is : 0.366 

distance 3
Working on tweet_dir
The score is : 0.502
[[ 24 192]
 [ 39 209]]
The F1 score is : 0.644 

distance 4
Working on tweet_dir
The score is : 0.504
[[ 25 191]
 [ 39 209]]
The F1 score is : 0.645 

distance 5
Working on tweet_dir
The score is : 0.5
[[ 24 192]
 [ 40 208]]
The F1 score is : 0.642 

distance 3
Working on tweet_volume
The score is : 0.504
[[  4 212]
 [ 18 230]]
The F1 score is : 0.667 

distance 4
Working on tweet_volume
The score is : 0.504
[[  7 209]
 [ 21 227]]
The F1 score is : 0.664 

distance 5
Working on tweet_volume
The score is : 0.511
[[  4 212]
 [ 15 233]]
The F1 score is : 0.672 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed

**Random Forest Approach**

In [12]:
#random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

features = [multifeat, vader,blob,tweetdir,tweet_vol,v_move,b_move]

feature = ['multifeat', 'vader','blob','tweetdir','tweet_vol','v_move','b_move']
for i in range(len(features)):
    scores = []
    mat = np.zeros([2,2])
    df_feature = pd.DataFrame(features[i])
    df_label = pd.DataFrame(diff)
    for j in range(1,10):
        print("Working on {} {} estimators".format(feature[i],j))
        #sbar = features[i].reshape(-1,1)

        if i == 0:
          #x_train, x_test, y_train, y_test = train_test_split(sbar, data["movement"], test_size=0.20, random_state=0)
          x_train, x_test, y_train, y_test = train_test_split(multifeat, df_label, test_size=0.20, random_state=0)
        else:
          x_train, x_test, y_train, y_test = train_test_split(df_feature, df_label, test_size=0.20, random_state=0)
        regressor = RandomForestClassifier(n_estimators=j*10, random_state=0,max_depth=i+2)
        regressor.fit(x_train, y_train)
        y_pred = regressor.predict(x_test)
        
        #predictions = knn.predict(x_test)
        score = regressor.score(x_test,y_test)
        #scores.append(score)
        print("Confusion Matrix is: \n{}".format(confusion_matrix(y_test,y_pred)))
        
        #print(classification_report(y_test,y_pred))
        print("Accuracy Score is: {}\n".format(round(accuracy_score(y_test, y_pred), 3)))
        f1 =  metrics.f1_score(y_test,y_pred)
        print("The F1 score is : {} \n".format(round(f1, 3)))
        #print(scores)
        #print(mat)
    #print("The score is : {}".format(score))
    #cm = metrics.confusion_matrix(y_test,predictions)
    #print(cm)

Working on multifeat 1 estimators
Confusion Matrix is: 
[[  2 214]
 [  4 244]]
Accuracy Score is: 0.53

The F1 score is : 0.691 

Working on multifeat 2 estimators
Confusion Matrix is: 
[[  6 210]
 [  6 242]]
Accuracy Score is: 0.534

The F1 score is : 0.691 

Working on multifeat 3 estimators
Confusion Matrix is: 
[[  4 212]
 [  5 243]]
Accuracy Score is: 0.532

The F1 score is : 0.691 

Working on multifeat 4 estimators


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Confusion Matrix is: 
[[  4 212]
 [  7 241]]
Accuracy Score is: 0.528

The F1 score is : 0.688 

Working on multifeat 5 estimators
Confusion Matrix is: 
[[  3 213]
 [  7 241]]
Accuracy Score is: 0.526

The F1 score is : 0.687 

Working on multifeat 6 estimators


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Confusion Matrix is: 
[[  3 213]
 [  7 241]]
Accuracy Score is: 0.526

The F1 score is : 0.687 

Working on multifeat 7 estimators
Confusion Matrix is: 
[[  3 213]
 [  7 241]]
Accuracy Score is: 0.526

The F1 score is : 0.687 

Working on multifeat 8 estimators


...

Confusion Matrix is: 
[[ 25 191]
 [ 18 230]]
Accuracy Score is: 0.55

The F1 score is : 0.688 

Working on b_move 8 estimators


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Confusion Matrix is: 
[[ 25 191]
 [ 17 231]]
Accuracy Score is: 0.552

The F1 score is : 0.69 

Working on b_move 9 estimators


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Confusion Matrix is: 
[[ 25 191]
 [ 17 231]]
Accuracy Score is: 0.552

The F1 score is : 0.69 

